In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import numpy as np
import mne
import torch.optim as optim
from sklearn.metrics import accuracy_score
from epochsMaker import import_EEG, EEG_to_epochs
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs

# EEG 데이터 가져오기 및 전처리
file_name = '[CGL]VI_12.txt'
eeg_array, label_array = import_EEG(file_name)
epochs = EEG_to_epochs(eeg_array, label_array)

event_id = {'Rest': 0, 'Right Hand': 1, 'Left Hand': 2, 'Feet': 3}  
events = epochs.events
event_name_map = {code: name for name, code in event_id.items()}

epochs_filtered = epochs.copy().filter(l_freq=0.5, h_freq=40.0)
filt_raw = epochs_filtered.copy().filter(l_freq=1.0, h_freq=None)

ica = ICA(n_components=15, max_iter="auto", random_state=97)
ica.fit(epochs_filtered)
ica.exclude = [1]
reconst_raw = epochs_filtered.copy()
ica.apply(reconst_raw)

data_3d = reconst_raw.get_data()  
print(data_3d.shape)
labels = epochs.events[:, -1]


# 테스트 데이터 가져오기 및 전처리
test_file_name = '[CGL]VI_11.txt'
test_eeg_array, test_label_array = import_EEG(test_file_name)
test_epochs = EEG_to_epochs(test_eeg_array, test_label_array)

test_epochs_filtered = test_epochs.copy().filter(l_freq=0.5, h_freq=40.0)
test_filt_raw = test_epochs_filtered.copy().filter(l_freq=1.0, h_freq=None)

test_ica = ICA(n_components=15, max_iter="auto", random_state=97)
test_ica.fit(test_epochs_filtered)
test_ica.exclude = [1]
test_reconst_raw = test_epochs_filtered.copy()
test_ica.apply(test_reconst_raw)

test_data_3d = test_reconst_raw.get_data()
test_labels = test_epochs.events[:, -1]


# ShallowConvNet 모델 정의
class ShallowConvNet(nn.Module):
    def __init__(self, num_channels, output_dim=4, dropout_prob=0.3):
        super(ShallowConvNet, self).__init__()

        self.conv_temp = nn.Conv2d(1, 40, kernel_size=(1, 25)) 
        self.conv_spat = nn.Conv2d(40, 40, kernel_size=(num_channels, 1), bias=False) 
        self.batchnorm1 = nn.BatchNorm2d(40, momentum=0.1, affine=True, eps=1e-5)
        self.avgpool1 = nn.AvgPool2d(kernel_size=(1, 75), stride=(1, 15)) 
        self.dropout1 = nn.Dropout(p=dropout_prob)
        self.fc = nn.Linear(self._calc_fc_input_dim(num_channels), output_dim)

    def _calc_fc_input_dim(self, num_channels):
        x = torch.zeros((1, 1, num_channels, 6000))  # 실제 데이터와 유사한 크기
        x = self.conv_temp(x)
        x = self.conv_spat(x)
        x = self.batchnorm1(x)
        x = torch.square(x)
        x = self.avgpool1(x)
        #return x.view(1, -1).size(1)  # fc 레이어 입력 크기 
        return 5080

    def forward(self, input):
        if len(input.shape) == 3:
            input = input.unsqueeze(1)
        
        x = self.conv_temp(input)
        x = self.conv_spat(x)
        x = self.batchnorm1(x)
        x = torch.square(x)
        x = self.avgpool1(x)
        x = torch.log(torch.clamp(x, min=1e-6))
        x = self.dropout1(x)
        x = x.view(x.size(0), -1)  # Flatten
        output = self.fc(x)
        
        return output

# 데이터로더 준비
def prepare_dataloader(data, labels, batch_size=32):
    dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(labels, dtype=torch.long))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

# 모델 학습 함수
def train_model(model, dataloader, num_epochs=40, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        running_loss = 0.0
        all_preds = []
        all_labels = []

        for inputs, labels in dataloader:
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        epoch_loss = running_loss / len(dataloader)
        epoch_acc = accuracy_score(all_labels, all_preds)
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

    print('Finished Training')

# 실제 학습 시작
num_channels = 31
output_dim = 4
dropout_prob = 0.3
batch_size = 8
num_epochs = 100
learning_rate = 0.001

dataloader = prepare_dataloader(data_3d, labels, batch_size=batch_size)
test_dataloader = prepare_dataloader(test_data_3d, labels, batch_size=batch_size)
model = ShallowConvNet(num_channels, output_dim, dropout_prob)

train_model(model, dataloader, num_epochs=num_epochs, learning_rate=learning_rate)



Not setting metadata
60 matching events found
No baseline correction applied
0 projection items activated
Setting up band-pass filter from 0.5 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (6.602 s)



C:\Users\SHR\AppData\Local\Temp\ipykernel_20116\4091907367.py:21: RuntimeWarning: filter_length (3301) is longer than the signal (2000), distortion is likely. Reduce filter length or filter a longer signal.
  epochs_filtered = epochs.copy().filter(l_freq=0.5, h_freq=40.0)
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 161 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 287 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 647 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 881 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1151 tasks      | elapsed:    0.3s


Setting up high-pass filter at 1 Hz


[Parallel(n_jobs=1)]: Done 1457 tasks      | elapsed:    0.5s
[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.6s



FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 1651 samples (3.302 s)



...

Fitting ICA to data using 31 channels (please be patient, this may take a while)
Selecting by number: 15 components


[Parallel(n_jobs=1)]: Done 1799 tasks      | elapsed:    0.4s


Fitting ICA took 1.1s.
Applying ICA to Epochs instance
    Transforming to ICA space (15 components)
    Zeroing out 1 ICA component
    Projecting back using 31 PCA components


C:\Users\SHR\AppData\Local\Temp\ipykernel_20116\4091907367.py:49: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  test_data_3d = test_reconst_raw.get_data()


Epoch [1/100], Loss: 40.3871, Accuracy: 0.2500
Epoch [2/100], Loss: 27.2968, Accuracy: 0.2000
Epoch [3/100], Loss: 19.1039, Accuracy: 0.2833
Epoch [4/100], Loss: 21.2251, Accuracy: 0.2000
Epoch [5/100], Loss: 16.2231, Accuracy: 0.1500
Epoch [6/100], Loss: 7.7877, Accuracy: 0.3333
Epoch [7/100], Loss: 5.8888, Accuracy: 0.2333
Epoch [8/100], Loss: 8.7652, Accuracy: 0.3167
Epoch [9/100], Loss: 9.5841, Accuracy: 0.1667
Epoch [10/100], Loss: 11.1542, Accuracy: 0.3167
Epoch [11/100], Loss: 9.5350, Accuracy: 0.2000
Epoch [12/100], Loss: 5.1195, Accuracy: 0.2333
Epoch [13/100], Loss: 8.4004, Accuracy: 0.2333
Epoch [14/100], Loss: 7.2591, Accuracy: 0.3333
Epoch [15/100], Loss: 12.0809, Accuracy: 0.1833
Epoch [16/100], Loss: 11.0835, Accuracy: 0.2667
Epoch [17/100], Loss: 11.0149, Accuracy: 0.2500
Epoch [18/100], Loss: 6.9594, Accuracy: 0.3000
Epoch [19/100], Loss: 9.5273, Accuracy: 0.3167
Epoch [20/100], Loss: 6.4874, Accuracy: 0.2500
Epoch [21/100], Loss: 8.5439, Accuracy: 0.2833
Epoch [22/100

In [2]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
import numpy as np
import matplotlib.pyplot as plt
from torchmetrics import Accuracy
#from getOtherData2 import get_data_2a

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
import seaborn as sns

In [3]:
from torchmetrics import Accuracy
from collections import deque

class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def evaluate(model, test_loader, loss_fn):
    model.eval()  # 모델을 평가 모드로 전환
    loss_test = AverageMeter()  # 손실을 추적하기 위한 AverageMeter 인스턴스 생성
    acc_test = Accuracy(task="multiclass", num_classes=4).to(device)  # 정확도를 계산하기 위한 Accuracy 인스턴스 생성

    all_preds = deque()
    all_labels = deque()

    with torch.no_grad():  # 평가 모드에서는 기울기를 계산하지 않도록 no_grad 블록 사용
        for inputs, targets in test_loader:  # 테스트 데이터 로더에서 입력과 타겟 데이터를 가져옴
            inputs, targets = inputs.to(device), targets.to(device)  # 데이터를 장치(CPU 또는 GPU)로 이동
            outputs = model(inputs)  # 모델을 사용하여 예측값 계산
            loss = loss_fn(outputs, targets)  # 예측값과 타겟을 사용하여 손실 계산

            loss_test.update(loss.item(), inputs.size(0))  # 손실 값을 업데이트
            acc_test.update(outputs, targets)  # 정확도 업데이트

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(targets.cpu().numpy())

    return loss_test.avg, acc_test.compute().item() #list(all_preds), list(all_labels)  # 평균 손실, 정확도, 예측값 및 실제 라벨 반환


In [4]:
# Evaluate the model using E data
loss_test, acc_test = evaluate(model, test_dataloader, loss_fn)
print(f'E Test Loss after: {loss_test:.4f}, E Test Accuracy after: {acc_test:.4f}')
loss_test, acc_test = evaluate(model, dataloader, loss_fn)
print(f'T Train Loss after: {loss_test:.4f}, T Train Accuracy after: {acc_test:.4f}')

NameError: name 'loss_fn' is not defined